### Function: `clean_goodrx_data`

Cleans and standardizes the raw scraped GoodRx data for analysis.

**Purpose:**
- Remove formatting artifacts from pharmacy names.
- Convert price columns to numeric values, handling missing or `"N/A"` entries.
- Replace placeholder text (`"N/A"`) in drug-related columns with NaN.
- Normalize special offer text to lowercase for consistency.
- Flag online pharmacies based on keyword detection.
- Map ZIP codes to corresponding city names for easier location-based analysis.

**Parameters:**
- `raw_csv_path` *(str)*: Path to the raw scraped CSV file.
- `save_cleaned_csv_path` *(str, optional)*: If provided, saves the cleaned DataFrame to this path.

**Returns:**
- **DataFrame**: Cleaned dataset ready for analysis and visualization.


In [1]:
def clean_goodrx_data(raw_csv_path, save_cleaned_csv_path=None):
    import pandas as pd
    import numpy as np

    # ZIP code to city mapping (update these as needed)
    zip_to_city = {
        '10001': 'New York',    # Manhattan, NY
        '90001': 'Los Angeles', # LA, CA
        '60601': 'Chicago',     # Chicago, IL
        # Add more ZIPs and cities here if needed
    }

    df = pd.read_csv(raw_csv_path)

    # All your previous cleaning steps ...
    df['pharmacy'] = df['pharmacy'].str.replace('\n', ' ', regex=True).str.strip()
    price_columns = ['price', 'standard_coupon_price', 'special_coupon_price']
    for col in price_columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('$', '', regex=False)
            .replace({'N/A': np.nan, 'nan': np.nan})
            .astype(float)
        )
    text_columns = ['drug', 'dosage', 'quantity']
    for col in text_columns:
        df[col] = df[col].replace('N/A', np.nan)
    df = df.dropna(subset=['drug'])
    df['special_offer'] = df['special_offer'].str.lower().str.strip()
    df['online_pharmacy'] = df['pharmacy'].str.contains('pay online', case=False, na=False)

    # --- NEW STEP: Assign city from ZIP code ---
    # If your df has a column 'location' or 'zip', make sure to use the correct one!
    # Here I'm using 'location' as the ZIP code column name. Update if needed.
    df['zip_str'] = df['location'].astype(str).str.zfill(5)
    df['city'] = df['zip_str'].map(zip_to_city)
    df = df.drop(columns=['zip_str'])  # clean up helper col

    # Save cleaned CSV
    if save_cleaned_csv_path:
        df.to_csv(save_cleaned_csv_path, index=False)
        print(f"Cleaned data saved to: {save_cleaned_csv_path}")
    return df


### Running the Data Cleaning Process

This step calls the `clean_goodrx_data()` function to:
1. Load the raw scraped data from `scraped_goodrx_data.csv`.
2. Apply all cleaning and transformation steps (price formatting, missing value handling, city mapping, etc.).
3. Save the cleaned dataset as `goodrx_cleaned_data.csv` for use in further analysis.
4. Store the cleaned DataFrame in `cleaned_df` for immediate in-notebook use.


In [4]:

cleaned_df = clean_goodrx_data(
    raw_csv_path='scraped_goodrx_data.csv',
    save_cleaned_csv_path='goodrx_cleaned_data.csv'
)


Cleaned data saved to: goodrx_cleaned_data.csv
